In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import json
import time
import datetime
import sys
import folium

In [2]:
pathData=os.path.join('C:\\','Users','kevin.ouaba','Desktop','journaux','Colissimo')
data = pd.read_hdf(r'C:\Users\kevin.ouaba\Desktop\journaux\Colissimo\Colis_Legacy_semaine14_PFC_TG2_Lyon_V2.h5',sep=';')

On crée un dico pour relier un sous traitants a une plateforme TG1 +
Fin de la data préparation (avec suppression des produits à exclure de l'analyse)

In [3]:
sous_traitants = pd.read_csv(os.path.join(pathData,'correspondance_TG1_sous_traitants.csv'),sep=";")
sous_traitants_dico = {}
for i in np.arange(len(sous_traitants['Site TG1'])):
    sous_traitants_dico[str(sous_traitants.loc[i,'Site TG1'])] = str(sous_traitants.loc[i,'TG1_Entite'])
list_pfc = [270070,310060,336130,354050,411380,556140,597250,621570,636540,670960,694710,774060,841736,911260,921770]
site_code_to_keep = list(sous_traitants_dico.keys())
site_code_to_keep = [int(x) for x in site_code_to_keep]
#Liste des sites codes des PFC et sous traitants
for ent in list_pfc:
    site_code_to_keep.append(ent)
#Il faut supprimer le site code de la PFC de st laurent de mure pour etudier tous les colis qui étaient ailleurs le premier jour
#Il faut enlever les sous traitants de la PFC de st laurent de mure
site_code_to_keep = list(set(site_code_to_keep))
site_code_to_keep.remove(694710)
site_code_to_keep.remove(382519)
site_code_to_keep.remove(690796)

In [4]:
product_to_supress = ['CA','CB','CC','CD','CE','CG','CH','CI','CJ','CK','CL','CM','CO','CP','CQ','CS','CT','CU','CW','CX','CY','CZ','EH','EO','EW','EX','EY','LC','ME','MF','MM','MN','MP','TZ','VD',
'VP','WP','XA','XE','XM','XP','XY','YE','YP','ZA','ZD','ZE','ZY','ZZ','0A','5W','6P','6X','7B','7C','7E','7F','7G','7H','7I','7J','7K','7M','7O','7P','7Q','7R','7T','7Z','8B','8C','8E',
'8F','8I','8M','8Q','8S','8T','8W','8Y','8Z','9B','9D','9W']
data = data.loc[~data['product_reference'].isin(product_to_supress),:]

In [6]:
print("Ajout de la famille d'évènement")
codes_evenements = pd.read_csv(r'C:\Users\kevin.ouaba\Desktop\journaux\Colissimo\codes_evenements.csv', encoding = 'latin-1', sep = ';',
                              usecols = ['Code', 'Typologie client'])\
                        .rename(columns = {'Code' : 'event_status_code_reasons1',
                                          'Typologie client' : 'Evt_typologie_client'})
    
data = data.merge(codes_evenements, how = 'left', on = 'event_status_code_reasons1')

Ajout de la famille d'évènement


Résultat général de la cohorte du 03/04

In [4]:
data['event_day_ref'] = data['event_day']
data.loc[(data['event_date'].dt.hour * 60 + data['event_date'].dt.minute).between(1, 299),
            'event_day_ref'] -= datetime.timedelta(days = 1)
data_04_03 = data.loc[(data['event_day_ref']==datetime.datetime(2017, 4, 3)) &
                      (data['Famille_evt']=='RI'),:]
#print(data_04_03['parcel_reference'].nunique())
#data_04_03 = data_04_03.loc[data_04_03['Flag_EDI_au_moins_1']==0,:]
print(data_04_03['parcel_reference'].nunique())
data_04_03 = data_04_03.loc[data_04_03['Flag_PFC_hors_mure']==1,:]
print(data_04_03['parcel_reference'].nunique())
list_parcel = data_04_03['parcel_reference'].unique()
print('Pourcentage d arrivé des colis les jours suivants')

grouped = data.loc[(data['parcel_reference'].isin(list_parcel)) & (data['Flag_PFC_mure']==1) & (data['Famille_evt']!='EDI'),['parcel_reference' ,'event_day_ref']].groupby('parcel_reference')
print(grouped.agg(lambda x: min(x))['event_day_ref'].value_counts() / len(list_parcel))
print('% colis non flashés à St Laurent de Mure - appelés colis perdus : ')
print((len(list_parcel) - len(grouped.agg(lambda x: min(x))['event_day_ref']))/len(list_parcel))

125968
81963
Pourcentage d arrivé des colis les jours suivants
2017-04-04    0.976282
2017-04-05    0.010712
2017-04-03    0.000976
2017-04-06    0.000915
2017-04-07    0.000183
2017-04-08    0.000024
Name: event_day_ref, dtype: float64
% colis non flashés à St Laurent de Mure - appelés colis perdus : 
0.01090736063833681


Fontion pour générer l'arbre de suivi des colis

In [5]:
def branche_arbre(dataset, list_colis_flash_post_date, list_colis_perdus, date,verbose):
    date_jm_1 = date - datetime.timedelta(days = 1)
    tableau_recap = pd.DataFrame(index = ['Livre le 0' + str(date_jm_1.day) + '/04 sans flash TG2',
                      'Livre le 0' + str(date.day) + '/04 sans flash TG2',
                      'Colis flashé St laurent de mure en TG2 le ' + str(date.day),
                      'Colis avec changement de site entre le ' + str(date_jm_1.day) + ' et le  ' + str(date.day)+ '/04',
                      'Colis nouvel evenement TG1 ' + str(date.day) + '/04',
                      'Colis sans evenement le ' + str(date.day) + '/04',
                      'Flashé ST LAURENT DE MURE le ' + str(date_jm_1.day) + ' en TG2',
                      'Colis_a_analyser_prochaine_etape',
                       'Colis_couvert'],
             columns = ['Vue au ' + str(date.day) + '/04'])
    grouped = data.loc[(data['parcel_reference'].isin(list_parcel)) & 
                       (data['Flag_PFC_mure']==1) & (data['Famille_evt']!='EDI'),['parcel_reference',
                                                    'event_day_ref']].groupby('parcel_reference')
    if verbose:
        print('Nombre de colis a St Laurent de Mur le ' + str(date_jm_1.day) + '/04 : ')
    grouped_bis = grouped.agg(lambda x: min(x))['event_day_ref']
    if verbose:
        print(len(grouped_bis.loc[grouped_bis==date_jm_1]))
    tableau_recap.iloc[6,0] = len(grouped_bis.loc[grouped_bis==date_jm_1])
    tableau_recap.iloc[2,0] = len(grouped_bis.loc[grouped_bis==date])
    if verbose:
        print('Etude des colis qui seront flashés à St Laurent de Mure')
        print('Nbre de colis avec evenement le 04/04 ')
    number_colis_event_04 = data.loc[(data['parcel_reference'].isin(list_parcel_flash_post_04)) & 
             (data['event_day_ref']==date),'parcel_reference'].nunique()
    if verbose:
        print(number_colis_event_04)
        print('Nbre de colis sans evenement le 04/04 ')
        print(len(list_parcel_flash_post_04) - number_colis_event_04)
    tableau_recap.iloc[5,0] = len(list_parcel_flash_post_04) - number_colis_event_04
    # Maintenant on s'interesse aux colis avec EVENT
    
    data_position_04_04 = data.loc[(data['parcel_reference'].isin(list_parcel_flash_post_04)) & 
                               (data['event_day_ref']==date),
         ['parcel_reference','site_code']]
    data_position_04_04.columns = ['parcel_reference','site_code_04']

    data_position_04_03 = data.loc[(data['parcel_reference'].isin(list_parcel_flash_post_04 )) & 
                                   (data['event_day_ref']==date_jm_1) &
                                   (data['site_code'].isin(site_code_to_keep)),
             ['parcel_reference','site_code']]
    data_position_04_03.columns = ['parcel_reference','site_code_03']

    data_position_04_04 = data_position_04_04.merge(data_position_04_03,left_on='parcel_reference',right_on='parcel_reference')
    if verbose:
        print('Nombre de colis qui ont changé de site code entre 03 et 04')
        print(data_position_04_04.loc[data_position_04_04['site_code_03'] != data_position_04_04['site_code_04'],
                                      'parcel_reference'].nunique())
    tableau_recap.iloc[3,0] = data_position_04_04.loc[data_position_04_04['site_code_03'] != data_position_04_04['site_code_04'],
                                  'parcel_reference'].nunique()
    if verbose:
        print('Nombre de colis qui n ont pas changé de site code entre 03 et 04')
        print(number_colis_event_04 - data_position_04_04.loc[data_position_04_04['site_code_03'] != data_position_04_04['site_code_04'],
                                      'parcel_reference'].nunique())
    tableau_recap.iloc[4,0] = number_colis_event_04 - data_position_04_04.loc[data_position_04_04['site_code_03'] != data_position_04_04['site_code_04'],
                                  'parcel_reference'].nunique()
    
    if verbose:
        print('************************')
        print('Etude des colis perdus')
    
    list_perdus = list_colis_perdus
    
    #On s'interesse aux colis livrés - Typologie famille LIV - le 03/04 et on les enleve de la liste des perdus
    data_perdus_04 = data.loc[(data['parcel_reference'].isin(list_perdus)),:]
    data_perdus_04['parcel_reference'].nunique()
    if date==datetime.date(2017, 4, 4):
        if verbose:
            print('Nbre de colis livrés le 03/04 : ')
            print(data_perdus_04.loc[(data_perdus_04['Evt_typologie_client']=='LIV') & 
                               (data_perdus_04['event_day_ref']==date_jm_1),'parcel_reference'].nunique())
        tableau_recap.iloc[0,0] = data_perdus_04.loc[(data_perdus_04['Evt_typologie_client']=='LIV') & 
                           (data_perdus_04['event_day_ref']==date_jm_1),'parcel_reference'].nunique()
        list_parcel_colis_livre = data_perdus_04.loc[(data_perdus_04['Evt_typologie_client']=='LIV') & 
                           (data_perdus_04['event_day_ref']==date_jm_1),'parcel_reference'].unique()
        list_perdus = list(set(list_perdus) - set(list_parcel_colis_livre))

    #On s'interesse aux colis livrés - EVENT LIVCFM - le 04/04 et on les enleve de la liste des perdus
    data_perdus_04 = data.loc[(data['parcel_reference'].isin(list_perdus)),:]
    data_perdus_04['parcel_reference'].nunique()
    if verbose:
        print('Nbre de colis livrés le ' + str(date.day) + '/04 : ')
        print(data_perdus_04.loc[(data_perdus_04['Evt_typologie_client']=='LIV') & 
                           (data_perdus_04['event_day']==date),
                                 'parcel_reference'].nunique())
    tableau_recap.iloc[1,0] = data_perdus_04.loc[(data_perdus_04['Evt_typologie_client']=='LIV') & 
                       (data_perdus_04['event_day']==date),
                             'parcel_reference'].nunique()
    
    #Nouvelle liste avec uniquement les colis 'perdus' non livrés le 03 et le 04
    livres_04 = list(data_perdus_04.loc[(data_perdus_04['Evt_typologie_client']=='LIV') & 
                       (data_perdus_04['event_day']==date),
                                        'parcel_reference'].unique())
    list_perdus = list(set(list_perdus) - set(livres_04))
    
    # Etude des colis "perdus" non livrés le 03 et 04 pour déterminer les éventuels changement de site
    data_perdus_04 = data.loc[(data['parcel_reference'].isin(list_perdus)) & 
                              (data_perdus_04['event_day_ref']==date),:]
    if verbose:
        print('Nbre de colis avec event le ' + str(date.day) + '/04 : ')
        print(data_perdus_04['parcel_reference'].nunique())
    list_parcel_event_04 = list(data_perdus_04['parcel_reference'].unique())
    if verbose:
        print('Nbre de colis sans event le ' + str(date.day) + '/04 : ')
        print(len(list_perdus) - data_perdus_04['parcel_reference'].nunique())
    tableau_recap.iloc[5,0] += len(list_perdus) - data_perdus_04['parcel_reference'].nunique()
    
    # On récupère la liste des colis avec event le 04/04 pour étudier leur changement de site 
    parcel_perdus_event_04 = list(data_perdus_04['parcel_reference'].unique())
    data_position_04_03 = data.loc[(data['parcel_reference'].isin(parcel_perdus_event_04)) & 
                                   (data['event_day_ref']==date_jm_1) &
                                   (data['site_code'].isin(site_code_to_keep)),
             ['parcel_reference','site_code']]
    data_position_04_03.columns = ['parcel_reference','site_code_03']

    data_position_04_04 = data.loc[(data['parcel_reference'].isin(parcel_perdus_event_04)) & 
                                   (data['event_day_ref']==date),
             ['parcel_reference','site_code']]
    data_position_04_04.columns = ['parcel_reference','site_code_04']

    data_position_04_04 = data_position_04_04.merge(data_position_04_03,
                                                    left_on='parcel_reference',
                                                    right_on='parcel_reference')

    if verbose:
        print('Nombre de colis qui ont changé de site code entre '+ str(date_jm_1.day) +' et '+ str(date.day))
        print(data_position_04_04.loc[data_position_04_04['site_code_03'] != data_position_04_04['site_code_04'],
                                      'parcel_reference'].nunique())
    tableau_recap.iloc[3,0] += data_position_04_04.loc[data_position_04_04['site_code_03'] != data_position_04_04['site_code_04'],
                                  'parcel_reference'].nunique() 
    if verbose:
        print('Nombre de colis qui n ont pas changé de site code entre '+ str(date_jm_1.day) + ' et ' + str(date.day))
        print(len(parcel_perdus_event_04) - data_position_04_04.loc[data_position_04_04['site_code_03'] != data_position_04_04['site_code_04'],
                                      'parcel_reference'].nunique())
    tableau_recap.iloc[4,0] += len(parcel_perdus_event_04) - data_position_04_04.loc[data_position_04_04['site_code_03'] != data_position_04_04['site_code_04'],
                                  'parcel_reference'].nunique()
    tableau_recap.iloc[7,0] = tableau_recap.iloc[3,0] + tableau_recap.iloc[4,0] + tableau_recap.iloc[5,0]
    tableau_recap.iloc[8,0] = tableau_recap.iloc[7,0] + tableau_recap.iloc[1,0] + tableau_recap.iloc[2,0]
    print(tableau_recap)
    return tableau_recap

Paramètre de la cellule suivante :

In [6]:
#la range correspond au nombre de jour pour lesquels on veut faire l'étude
# data = dataset des évènements
# list_parcel_flash_post_04 = List des ID colis qui seront flashés en TG2 à St Laurent de Mure à partir du 04
# list_perdus = Liste des ID colis "égarés" (c'est à dire ils ont été flashé en TG1 mais on ne les retrouve pas 
# en TG2 durant la semaine
# verbose = True pour avoir les détails des calculs, et false pour n'avoir que le résultat

In [8]:
date_to_start = datetime.date(2017,4,4)
grouped_bis = grouped.agg(lambda x: min(x))['event_day_ref']
colis_trouves = list(grouped_bis.index)
grouped_bis = grouped_bis.loc[grouped_bis > datetime.datetime(2017, 4, 4)]
list_parcel_flash_post_04 = list(grouped_bis.index)
s_list_parcel = set(list(list_parcel))
s_colis_trouve = set(colis_trouves)
list_perdus = s_list_parcel - s_colis_trouve
for i in range(0,1):
    print(' ')
    print('*********************')
    print(date_to_start + datetime.timedelta(i))
    date_to_analyze = date_to_start + datetime.timedelta(days=i)
    branche_arbre(data,list_parcel_flash_post_04,list_perdus,date_to_analyze,verbose=False)
    if date_to_analyze == datetime.date(2017,4,4):
        list_livre_03 = list(data.loc[(data['Evt_typologie_client']=='LIV') & 
                   (data['event_day']==(date_to_analyze - datetime.timedelta(1))) & 
                    (data['parcel_reference'].isin(list_perdus))
               ,'parcel_reference'].unique())
        list_livre_04 = list(data.loc[(data['Evt_typologie_client']=='LIV') & 
                   (data['event_day']==date_to_analyze) & 
                    (data['parcel_reference'].isin(list_perdus))
               ,'parcel_reference'].unique())
        list_perdus = list(set(list_perdus)-set(list_livre_03))
        list_perdus = list(set(list_perdus)-set(list_livre_04))
    list_livre_03 = list(data.loc[(data['Evt_typologie_client']=='LIV') & 
                   (data['event_day']==date_to_analyze) & 
                    (data['parcel_reference'].isin(list_perdus))
               ,'parcel_reference'].unique())
    list_perdus = list(set(list_perdus)-set(list_livre_03))
    grouped_bis = grouped.agg(lambda x: min(x))['event_day_ref']
    grouped_bis = grouped_bis.loc[grouped_bis > (date_to_analyze + datetime.timedelta(1))]
    list_parcel_flash_post_04 = list(grouped_bis.index)

 
*********************
2017-04-04
                                                   Vue au 4/04
Livre le 03/04 sans flash TG2                                6
Livre le 04/04 sans flash TG2                              105
Colis flashé St laurent de mure en TG2 le 4              80019
Colis avec changement de site entre le 3 et le ...         569
Colis nouvel evenement TG1 4/04                             61
Colis sans evenement le 4/04                              1123
Flashé ST LAURENT DE MURE le 3 en TG2                       80
Colis_a_analyser_prochaine_etape                          1753
Colis_couvert                                            81877


In [25]:
##### Etude des return to sender
print('Nombre de colis perdus qui ont été renvoyés à l\'expediteur : ' + str(data.loc[(data['parcel_reference'].isin(list_perdus)) & 
        (data['return_to_sender']=='O'),'parcel_reference'].nunique()) + '/' + str(len(list_perdus)))

Nombre de colis perdus qui ont été renvoyés à l'expediteur : 259/363


In [77]:
list_real_perdus = list(set(list_perdus) - set(list(data.loc[(data['parcel_reference'].isin(list_perdus)) & 
        (data['return_to_sender']=='O'),'parcel_reference'].unique())))

In [78]:
len(list_real_perdus)

581

FIN

In [12]:
grouped = data.loc[(data['parcel_reference'].isin(list_parcel)) & (data['Flag_PFC_mure']==1) & (data['Famille_evt']!='EDI'),
                   ['parcel_reference' ,'event_day_ref']].groupby('parcel_reference')
print(grouped.agg(lambda x: min(x))['event_day_ref'].value_counts() / len(list_parcel))
print('% colis non flashés à St Laurent de Mure - appelés colis perdus : ')
print((len(list_parcel) - len(grouped.agg(lambda x: min(x))['event_day_ref']))/len(list_parcel))

2017-04-04    0.976282
2017-04-05    0.010712
2017-04-03    0.000976
2017-04-06    0.000915
2017-04-07    0.000183
2017-04-08    0.000024
Name: event_day_ref, dtype: float64
% colis non flashés à St Laurent de Mure - appelés colis perdus : 
0.01090736063833681


In [16]:
grouped_agg = grouped.agg(lambda x: min(x))['event_day_ref']

In [28]:
list_flash_04 = list(grouped_agg.loc[grouped_agg==datetime.date(2017,4,4)].index)
list_flash_05 = list(grouped_agg.loc[grouped_agg==datetime.date(2017,4,5)].index)
list_flash_06 = list(grouped_agg.loc[grouped_agg==datetime.date(2017,4,6)].index)
list_flash_07 = list(grouped_agg.loc[grouped_agg==datetime.date(2017,4,7)].index)
list_flash_08 = list(grouped_agg.loc[grouped_agg==datetime.date(2017,4,8)].index)
list_perdus = list(set(list_parcel) - set(list(grouped_agg.index)))

In [33]:
data['announced_volume'] = data['announced_height'] * data['announced_width'] * data['announced_length']
data['measured_volume'] = data['measured_height'] * data['measured_width'] * data['measured_length']

In [38]:
data.loc[data['parcel_reference'].isin(list_parcel),['announced_height', 'announced_width',
       'announced_length','announced_volume' ,'measured_height', 'measured_width',
       'measured_length','measured_volume', 'announced_weight', 'measured_weight']].mean()

announced_height       12.006372
announced_width        23.326904
announced_length       33.003278
announced_volume    13293.029860
measured_height        12.006372
measured_width         23.326904
measured_length        33.003278
measured_volume     13293.029860
announced_weight        0.966752
measured_weight         1.507324
dtype: float64

In [44]:
data.loc[data['parcel_reference'].isin(list_flash_06),['announced_height', 'announced_width',
       'announced_length','announced_volume' ,'measured_height', 'measured_width',
       'measured_length','measured_volume', 'announced_weight', 'measured_weight']].mean()

announced_height       12.756329
announced_width        23.729430
announced_length       37.151899
announced_volume    21739.533228
measured_height        12.756329
measured_width         23.729430
measured_length        37.151899
measured_volume     21739.533228
announced_weight        0.700798
measured_weight         2.534527
dtype: float64

In [150]:
data.loc[(data['parcel_reference'].isin(list_flash_07)) & 
         (data['Famille_evt']=='RI') & (data['Flag_PFC_hors_mure']==1),['announced_height', 'announced_width',
       'announced_length','announced_volume' ,'measured_height', 'measured_width',
       'measured_length','measured_volume', 'announced_weight', 'measured_weight']].median()

announced_height       3.00
announced_width       20.00
announced_length      30.00
announced_volume    1080.00
measured_height        3.00
measured_width        20.00
measured_length       30.00
measured_volume     1080.00
announced_weight       0.00
measured_weight        0.12
dtype: float64

In [130]:
data.loc[(data['parcel_reference'].isin(list_real_perdus))& 
         (data['announced_weight']!=0) & (data['event_day_ref']==datetime.date(2017,4,3)) &
                (data['Famille_evt']=='RI'),['announced_height', 'announced_width',
       'announced_length','announced_volume' ,'measured_height', 'measured_width',
       'measured_length','measured_volume', 'announced_weight', 'measured_weight']]

,announced_height,announced_width,announced_length,announced_volume,measured_height,measured_width,measured_length,measured_volume,announced_weight,measured_weight
3122418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000,0.00
3122420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000,0.00
3122501,36.0,36.0,45.0,58320.0,36.0,36.0,45.0,58320.0,2.000,0.00
3122510,36.0,36.0,45.0,58320.0,36.0,36.0,45.0,58320.0,2.000,0.00
3122514,36.0,36.0,45.0,58320.0,36.0,36.0,45.0,58320.0,2.000,0.00
3122754,5.0,15.0,22.0,1650.0,5.0,15.0,22.0,1650.0,2.000,0.00
3122756,5.0,15.0,22.0,1650.0,5.0,15.0,22.0,1650.0,2.000,0.00
3122776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000,0.00
3122778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000,0.00
3122784,5.0,14.0,22.0,1540.0,5.0,14.0,22.0,1540.0,2.000,0.24


In [149]:
print('Taux d annonce sans poids : ')
print((data.loc[(data['parcel_reference'].isin(list_flash_07)) &
               (data['announced_weight']==0) & (data['event_day_ref']==datetime.date(2017,4,3)) &
                (data['Famille_evt']=='RI'),'parcel_reference'].nunique())/len(list_flash_07)) 

Taux d annonce sans poids : 
0.4666666666666667


In [66]:
print((data.loc[(data['parcel_reference'].isin(list_perdus)) &
               (data['announced_weight']==0) & (data['event_day_ref']==datetime.date(2017,4,3)) &
                (data['Famille_evt']=='RI'),'parcel_reference'].nunique())/len(list_perdus)) 

0.5794183445190156


In [74]:
data.loc[(data['parcel_reference'].isin(list_perdus))& 
         (data['announced_weight']!=0),['announced_height', 'announced_width',
       'announced_length','announced_volume' ,'measured_height', 'measured_width',
       'measured_length','measured_volume', 'announced_weight', 'measured_weight']]

,announced_height,announced_width,announced_length,announced_volume,measured_height,measured_width,measured_length,measured_volume,announced_weight,measured_weight
470875,36.0,52.0,70.0,131040.0,36.0,52.0,70.0,131040.0,8.630,8.85
3479071,8.0,21.0,24.0,4032.0,8.0,21.0,24.0,4032.0,0.550,0.45
2411096,8.0,23.0,38.0,6992.0,8.0,23.0,38.0,6992.0,0.680,0.69
960199,15.0,26.0,34.0,13260.0,15.0,26.0,34.0,13260.0,0.500,0.64
3114245,39.0,41.0,48.0,76752.0,39.0,41.0,48.0,76752.0,2.000,0.00
475160,36.0,52.0,70.0,131040.0,36.0,52.0,70.0,131040.0,8.630,8.85
3114500,39.0,41.0,48.0,76752.0,39.0,41.0,48.0,76752.0,2.000,0.00
478961,8.0,21.0,24.0,4032.0,8.0,21.0,24.0,4032.0,0.670,0.65
3116144,16.0,25.0,28.0,11200.0,16.0,25.0,28.0,11200.0,2.000,1.00
3301828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.416,0.55


In [148]:
data.loc[(data['parcel_reference'].isin(list_flash_07)) & (data['event_day_ref']==datetime.date(2017,4,3)) &
                (data['Famille_evt']=='RI') & 
         (data['measured_weight'].isnull()),'parcel_reference'].nunique() 

2

In [127]:
data.loc[(data['parcel_reference'].isin(list_real_perdus)) &
               (data['announced_weight']==0) & (data['event_day_ref']==datetime.date(2017,4,3)) &
                (data['Famille_evt']=='RI'),'measured_weight'].sort(inplace=False)

C:\Users\kevin.ouaba\AppData\Local\Continuum\Miniconda3\envs\journaux\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting
  app.launch_new_instance()


3123374     0.00
3123400     0.00
3126140     0.00
3126143     0.00
3132155     0.00
3132180     0.00
4157800     0.05
1668891     0.05
2463831     0.05
2257661     0.06
1915481     0.07
3412454     0.08
3413577     0.09
2462565     0.10
2264878     0.10
41936       0.10
4158070     0.10
1664954     0.10
31324       0.10
3564005     0.10
2259060     0.10
234934      0.11
3415864     0.12
4329501     0.12
1916736     0.13
1918357     0.13
30861       0.14
4158485     0.15
41450       0.15
31490       0.15
           ...  
3611515     5.00
3579327     5.00
3579455     5.05
3611530     5.05
2264421     5.22
2258540     5.22
2256179     5.70
1675395     6.00
3472629     6.22
2858826     6.85
3140429     7.60
1916414     7.76
1916617     8.48
1918571     8.48
2858404     8.76
2857450     8.94
2863570    11.04
2859900    11.04
2859148    11.14
3035342    11.98
3036424    11.98
3036967    11.98
2859336    14.50
3312289    16.56
3312303    16.72
3312285    16.96
3312359    18.30
2859222    23.

In [95]:
data.loc[data['announced_volume'] != data['measured_volume'],'parcel_reference'].nunique()

71592

In [108]:
len(list_flash_04)

80019

581